<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Applying AI Weather Models With NVIDIA Earth-2</h1>
<h2><b>Part 2:</b> Validation</h2>
<br>

In the second part of our workshop, we will have a closer look at the output data produced by FourCastNet (SFNO). When validating output from weather models, different applications have varying objectives. For example, applications in the Energy industry may be interested in forecasting wind power production for intra-day and day-ahead trading, while applications for agriculture may focus more on sub-seasonal to seasonal timeframes. We will explore a set of common evaluation metrics that can be used to understand how our forecasts add value. Keep in mind that we are only working on a tiny dataset for demonstration purposes, however. Rigorous validation would typically cover multiple start times and more ensemble members. 

In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

import torch
import xarray as xr

from earth2studio.data import NCAR_ERA5, WB2Climatology, fetch_data, prep_data_array
from earth2studio.statistics import acc, crps, lat_weight, rank_histogram, rmse, spread_skill_ratio
from plot import animate_global, animate_local, figure_global, plot_global_ensemble, plot_line_ensemble, plot_rank_examples, plot_rank_histograms, plot_score_line, plot_wind_power_output
from utils import get_lat_lon
from wind import calculate_power_output, get_wind_turbine_model

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Exploration

We will start by loading the forecast we produced earlier. Remember that we saved seven different variables in the previous part. Feel free to explore the other variables in any of the plots below.

In [ ]:
ds = xr.open_dataset("/workspace/exercises/output/part_01.nc", engine="netcdf4")

start_time = ds.time.values.squeeze()
print("Available variables: ", list(ds))

Let's get an overview by creating a global animation of 2-meter temperature. Creating the animation will take some time, but afterward, you can watch a short movie of your weather forecast. Note that with 6-hour intervals between time steps, the progression is not particularly smooth. Earth-2 also offers an AI interpolation model that provides additional time steps between the model outputs to achieve a higher temporal resolution.

In [ ]:
animate_global(ds.isel(ensemble=0).t2m, start_time, max_frames=5)

There is a lot going on on Earth, so let's zoom in on a smaller window. The output grid is 721x1440, so we can select latitudes and longitudes on a quarter-degree grid (i.e., in steps of 0.25). For the purpose of this tutorial, the coordinates for some larger cities were predefined and are available via `get_lat_lon`. Let's start by looking at the region around the Gulf of Mexico, which was heavily affected by Hurricane Beryl. Since we saved the u- and v-components of 10-meter wind, we can calculate the wind speed for each point by taking the Euclidian length (square root of the sum of squares). With this, we can clearly see the hurricane progressing toward the Southern US border.

In [ ]:
location = "Houston"
loc_lat, loc_lon = get_lat_lon(location)

In [ ]:
animate_local((ds.isel(ensemble=0).u10m ** 2 + ds.isel(ensemble=0).v10m ** 2) ** 0.5, start_time, loc_lat, loc_lon, cb_label="Wind speed [m/s]")

The plots above only show the output for the first ensemble member. We can get a better idea of the forecast uncertainty by looking at the differences between ensemble members. For this, we will first zoom in on a single grid cell. Each of the lines in the plots below shows the trajectory for one ensemble member. The initial perturbation is small compared to the spread of the ensemble at later lead times. This sensitivity to tiny changes in the initial conditions is why we say the weather system is [chaotic](https://en.wikipedia.org/wiki/Chaos_theory).

In [ ]:
plot_line_ensemble(ds.tcwv, start_time, loc_lat, loc_lon, "tcwv [kg/m²]")

The plots below show the bigger picture. The globes on the left show the output for three of the ensemble members at the last step of the forecast. The globe on the right shows the standard deviation of members, and we can identify regions of high and low variability.

In [ ]:
plot_global_ensemble(ds.tcwv, start_time, loc_lat, loc_lon, cb_label="tcwv [kg/m²]", ilead_time=10)

We may want to use our forecast to predict the power output of a wind park. Let's start smaller and calculate the expected power output of a wind turbine. For this, we will use `windpowerlib`, a Python library focused on wind power modeling. We can choose one of many predefined reference configurations and pass it to the library, which will handle the calculations based on our 10-meter and 100-meter wind speeds.

In [ ]:
# Here, we use the IEA 15 MW offshore reference turbine
# Supported turbine types can be found at https://openenergyplatform.org/dataedit/view/supply/wind_turbine_library
wind_turbine_specification = {
    "turbine_type": "IEA-15-240-RWT",  # turbine type as in register
    "hub_height": 150,  # in m
}
wt = get_wind_turbine_model(wind_turbine_specification)

The power output is calculated based on wind speeds extrapolated using a logarithmic profile with an estimated roughness length. The plots show the wind speed at hub height, the power curve of the wind turbine, and the predicted power output over the course of the forecast. Individual ensemble members are shown as dotted lines, while the solid lines indicate the ensemble mean.

In [ ]:
ds_power = calculate_power_output(ds, wt, loc_lat=65, loc_lon=23)
plot_wind_power_output(ds_power, wt, start_time=start_time)

## Scoring

To estimate the skill of our forecast, we will score the outputs against the official ERA5 reanalysis. Earth2Studio provides optimized routines to calculate scoring metrics, but first, we need to obtain the reference data for comparison. For this, we can use `fetch_data`, which was also used under the hood when we ran our ensemble workflow. Like the workflow, it requires a data source (we will again use `NCAR_ERA5`), a start time, the requested variables, and the lead times. For the workflow, we only needed a single time slice as initial conditions, but for scoring, we will use all 17 time slices to compare the skill over time.

In [ ]:
ncar_era5 = NCAR_ERA5(verbose=True)
scoring_variables = ["t2m", "u10m", "z500"]

x_true, coords_true = fetch_data(
    source=ncar_era5,
    time=[start_time],
    variable=scoring_variables,
    lead_time=ds.lead_time.values,
)

Earth2Studio provides the `prep_data_array` convenience function to convert xarray `DataArray`s to pure PyTorch tensors and the respective coords. With PyTorch tensors, we can leverage our GPU to accelerate the computation. The coords track the information for the dimensions.

In [ ]:
x_pred, coords_pred = prep_data_array(ds[scoring_variables].to_dataarray().transpose("ensemble", "time", "lead_time", ...))

Since we are working on a regular 721x1440 grid, we should weight down the points at the poles when calculating global validation scores. This is because the distance between individual points at higher latitudes is smaller than at the equator. Without weighting, we would give more importance to regions near the poles. We can use Earth2Studio's `lat_weight` to assign gradually smaller weights to polar regions.

In [ ]:
weights = torch.ones(len(ds.lon)) * lat_weight(torch.from_numpy(ds.lat.values)).unsqueeze(1)

figure_global(xr.DataArray(data=weights, coords={"lat": ds.lat.values, "lon": ds.lon.values}), *get_lat_lon("San Jose"), colorbar=True)

When scoring probabilistic ensembles, we use different metrics than when scoring deterministic forecasts. However, it is common practice to subject the ensemble mean to a deterministic validation, so this is what we will do first. One simple and commonly used deterministic scoring metric is the Root Mean Squared Error (RMSE), which calculates the squared difference between our prediction and the reference data, sums this difference for all datapoints, and then takes the square root. To calculate the RMSE, we simply pass the input tensors and our weights to the `rmse` scoring routine provided by Earth2Studio. We specify `lat` and `lon` as reduction dimensions, so we get back a single global value per variable and time step. This is why it is important to provide the latitude weights upfront. The scoring function also returns a PyTorch tensor and its coordinates, so we unpack the return value and let xarray construct a dataset from the results. This will make plotting easier.

In [ ]:
RMSE = rmse(reduction_dimensions=["lat", "lon"], weights=weights)
rmse_score = xr.DataArray(*RMSE(x_pred.mean(axis=0), {k: v for k, v in coords_pred.items() if k != "ensemble"}, x_true, coords_true)).to_dataset("variable")

In the plots below, we see that the RMSE is small for short lead times and then gradually increases further into the forecast. This is expected as uncertainties in the modeling process add up over time. The longer the lead time, the less accurate a forecast generally is.

In [ ]:
plot_score_line(rmse_score, start_time, ["t2m", "u10m", "z500"], ["RMSE [K]", "RMSE [m/s]", "RMSE [gpm]"])

The ensemble mean is usually a better estimator of the expected future than a single deterministic forecast. However, the ensemble is only meaningful if the deviation between ensemble members covers the entire spectrum of plausible future weather scenarios. If the members are too similar, we underestimate the occurrence of weather extremes. The ensemble is then called underdispersive. If the differences are too large, the ensemble is called overdispersive. How large should the spread between ensemble members be? One metric designed to answer this question is the Spread-Skill Ratio (SSR). It compares the spread (the standard deviation of the ensemble members) to the skill (the RMSE of the ensemble). Ideally, the ratio between the two would be 1. We can use `spread_skill_ratio` to compute the ratio, just as we used `rmse` for the RMSE. The only difference is that this time we also need to specify the name of the ensemble dimension, so the routine knows which dimension to calculate the mean and standard deviation across.

In [ ]:
SSR = spread_skill_ratio(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], reduction_weights=weights)
ssr = xr.DataArray(*SSR(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

The line plots below show the SSR for our three variables. Values larger than 1 mean that the ensemble is overdispersive and values below 1 mean it is underdispersive. Our SRR seems to be off for short lead times, which indicates that we should take time to tune our perturbation method. The two-part publication from LBNL and NIVIDA researchers mentioned earlier introduces a perturbation method based on bred vectors that achieves convincing results for perturbing AI weather forecasts ([part 1](https://arxiv.org/abs/2408.03100) and [part 2]( https://arxiv.org/abs/2408.01581v1)).

In [ ]:
plot_score_line(ssr, start_time, ["t2m", "u10m", "z500"], ["SSR", "SSR", "SSR"])

A second important validation strategy for probabilistic forecasts uses rank histograms. For these, we do not compute a single score but look at the rank of the validation data compared to our ensemble members. The rank is the position the validation value would take when ordering all ensemble values together with the validation value. For example, if there are 13 ensemble members that predict a value smaller than the validation, then the validation rank would be 14. We do this for every single prediction and then look at the histogram of these ranks.

Ideally, the ranks would be spread evenly, which indicates that the validation is indistinguishable from the ensemble predictions. For underdispersive ensembles, the rank of the validation is often very high or low, creating a U-shaped histogram. In the overdispersive case, the ranks will group in the middle because the ensemble predictions are more extreme (higher or lower) than the validation. We can also identify biases. A negative bias shows as an upward slope because the validation tends to get high ranks. A positive bias shows through a downward slope.

In [ ]:
plot_rank_examples()

Calculating rank histograms works like the other metrics. The difference now is that the resulting data contains information about the histogram bin centers and counts, which we can plot.

In [ ]:
RH = rank_histogram(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], number_of_bins=5)
rh = xr.DataArray(*RH(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

Compare the resulting histograms to the examples above. How do the different variables perform for different lead times?

In [ ]:
plot_rank_histograms(rh, start_time, ["t2m", "u10m", "z500"], appr_n=4)

Again, note that we calculated the statistics only for a single start time and a small ensemble. In practice, this evaluation would be carried out for many start times and/or longer forecast horizons and with larger ensembles. Also, it may make sense to use different types of validation data. For example, we used reanalysis data. Other options include actual observations, other forecast data, or climatologies.

When you are done with this notebook, please shut down the kernel (in the context menu at the top, choose "Kernel" and then "Shut Down Kernel"). Then proceed to the [notebook for part 3](./part_03_downscaling.ipynb) to learn about downscaling. If you have extra time on your hands, you can learn about two more common evaluation metrics below.

## Extra

Another popular evaluation metric for deterministic forecasts is the [Anomaly Correlation Coefficient](https://confluence.ecmwf.int/display/FUG/Section+6.2.2+Anomaly+Correlation+Coefficient) (ACC). It is the correlation of anomalies between a forecast and a reference, both relative to climatology. Its value lies between -1 and 1, with 1 designating perfect correlation. Earth2Studio provides easy access to climatology data from [WeatherBench 2](https://weatherbench2.readthedocs.io/en/latest/data-guide.html). We can simply pass the `WB2Climatology` data interface to the scoring routine, which will download and cache the required data automatically. Apart from specifying the climatology, the code is the same as for calculating the RMSE.

In [ ]:
ACC = acc(reduction_dimensions=["lat", "lon"], climatology=WB2Climatology(), weights=weights)
acc_score = xr.DataArray(*ACC(x_pred.mean(axis=0), {k: v for k, v in coords_pred.items() if k != "ensemble"}, x_true, coords_true)).to_dataset("variable")

Early in the forecast, our predictions have ACC values close to 1, indicating very good performance. Over the course of the forecast, the ACC slowly decreases. We can consider a forecast to be valuable for ACC values above 0.55-0.6 (according to, e.g., the [German Weather Service](https://www.dwd.de/EN/ourservices/seasonals_forecasts/anomaly_correlation.html) and [ECWMF](https://confluence.ecmwf.int/display/FUG/Section+6.2.2+Anomaly+Correlation+Coefficient)). This means our forecast remains skillful over the 96 hours we are considering for this evaluation.

In [ ]:
plot_score_line(acc_score, start_time, ["t2m", "u10m", "z500"], ["ACC", "ACC", "ACC"])

Another important evaluation metric for ensemble forecasts is the Continuous Ranked Probability Score (CRPS). It measures the difference between the forecasted probability distribution and the true outcome. The CRPS score is calculated as the integral of the area between the forecasted Cumulative Distribution Function (CDF) and the true CDF. A lower CRPS score indicates a better forecast. For a deterministic forecast, it defaults to the Mean Absolute Error (MAE). With Earth2Studio, we can calculate the CRPS just as we calculate the other metrics.

In [ ]:
CRPS = crps(ensemble_dimension="ensemble", reduction_dimensions=["lat", "lon"], weights=weights)
crps_score = xr.DataArray(*CRPS(x_pred, coords_pred, x_true, coords_true)).to_dataset("variable")

As expected, the CRPS is low at first and then gradually increases for longer lead times.

In [ ]:
plot_score_line(crps_score, start_time, ["t2m", "u10m", "z500"], ["CRPS", "CRPS", "CRPS"])